Aluno: Jhonata Ezequiel Alves de Miranda

Matrícula: 20190046473

1º) Utilizando a base de dados de https://www.kaggle.com/datasets/csafrit2/higher-education-students-performance-evaluation, elabore uma solução para identificar o OUTPUT Grade do estudante. Lembre-se de comentar seu código no notebook detalhadamente, explicando cada passo.

A) Faça o pré-processamento dos dados (limpeza, engenharia de variáveis, etc) e deixe os seus dados preparados para aplicar o modelo.
OBS: Utilize pelo menos uma forma de redimensionamento de atributos (selecionando ou agregando) e avalie o resultado de utilizar todas eles e essa amostra.

B) Faça uma análise exploratória dos dados de saída, utilizando box plot, mostrando a aplicação de técnicas de under ou oversampling para que as classes tenham o mesmo tamanho.

C) Utilize a biblioteca AUTOML para fazer a previsão. Para o melhor algoritmo, teste 3 variações de um dos seus hiperparâmetros.

D) Para avaliar os resultados, utilize e explique a matriz de confusão. Além disso, escolha 2 métricas de sua preferência e o que o seu resultado significa.


OBS: em ordem, os blocos de código 1, 2, e 4 precisarão ser executados para todas as questões. Os códigos estão enumerados por comentarios

In [ ]:
#1
#questão 1 letra A

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

base = pd.read_csv('student_prediction.csv')

In [ ]:
#2
#limpeza de dados

#eliminando todas as linhas com dados ausentes
base = base.dropna(how = 'any')

#eliminando as duplicatas
base = base.drop_duplicates()

Em seguida, farei a verificação de atributos para tentar ver se tem alguma correlação que impacte muito com a nota final

In [ ]:
#3
#verificando a correlação entre todos os atributos
corr = base.corr()
corr.style.background_gradient(cmap = 'coolwarm')

Sobre as colunas que foram retiradas: 

A princípio achei que as colunas referentes ao ambiente que o estudante vive seriam importantes, no entanto pessoas que tem problemas em suas residências normalmente são pessoas de classe baixa, e julguei que caso o estudante tivesse uma bolsa para estudar na universidade, ele provavelmente pertenceria a esse grupo.

In [ ]:
#4
#reduzindo a quantidade de atributos
base = base.drop(columns=['STUDENTID', 'COURSE ID', 'MOTHER_EDU', 'FATHER_EDU', 'MOTHER_JOB', 'FATHER_JOB', 'SALARY', '#_SIBLINGS', 'KIDS', 'TRANSPORT', 'IMPACT', 'ATTEND_DEPT'])

In [ ]:
#5
#Questão 1 Letra B
#verificando os dados de saída com boxplot

plt.boxplot(base['GRADE'])

plt.title('Estudantes')
plt.xlabel('Nota')
plt.ylabel('Valores')

In [ ]:
#6
#Verificando a média das notas

base['GRADE'].mean()

In [ ]:
#7
#Verificando a variância e o desvio padrão das notas

variancia = base['GRADE'].var()
desvio = base['GRADE'].std()

print('Variancia: ' + str(variancia)) 
print('Desvio padrao: ' + str(desvio))

In [ ]:
#8
#verificando se há um desbalanceamento entre as notas
base['GRADE'].value_counts()

Aplicando Oversampling:

In [ ]:
#9
#dividindo a base numa proporção 7 por 3
from sklearn.model_selection import train_test_split

#separando a classe alvo
X = base.drop('GRADE', axis = 1)
y = base['GRADE']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

In [ ]:
#10
#utilizando o método random para o oversampling
from imblearn.over_sampling import RandomOverSampler
oversampler = RandomOverSampler(random_state = 1)

X_smote, y_smote = oversampler.fit_resample(X_train, y_train)

y_smote.value_counts()

In [ ]:
#11
#juntando o X_smote (dataframe sem o atributo notas) com o y_smote (dataframe com as notas) para formar uma nova base
convertido = y_smote.to_frame() #transformando de series para dataframe

base_final = X_smote.join(convertido['GRADE'])

bloco 12:

Após rodas os dois comandos abaixo, reinicie o ambiente de execução

In [ ]:
!pip3 install auto-sklearn

In [ ]:
!pip3 install scipy -U

In [ ]:
#13
#Questão 1 Letra C
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from autosklearn.classification import AutoSklearnClassifier

In [ ]:
#14
#Dividindo a base após o oversampling (necessita do bloco 11)

#separando a classe alvo
X = base_final.drop('GRADE', axis = 1)
y = base_final['GRADE']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

In [ ]:
#15
#o algoritmo vai executar por 5 minutos, com um tempo limite de 30 segundos para cada instância diferente, usando todos os processadores disponíveis
modelo_holdout_normal = AutoSklearnClassifier(time_left_for_this_task=5*60, per_run_time_limit=30, n_jobs=-1)

#inicia a busca
modelo_holdout_normal.fit(X_train, y_train)

#mostrando as estatísticas do processo
print(modelo_holdout_normal.sprint_statistics())

#testando o melhor modelo
y_h1 = modelo_holdout_normal.predict(X_test)
acc = accuracy_score(y_test, y_h1)
print("Acuracia: %.3f" % acc)
print(modelo_holdout_normal.leaderboard())

In [ ]:
#16
#usando cross validation com 10 folds
modelo_cross = AutoSklearnClassifier(time_left_for_this_task=5*60, per_run_time_limit=30, n_jobs=-1, resampling_strategy = 'cv', resampling_strategy_arguments={'folds': 10})
#inicia a busca
modelo_cross.fit(X_train, y_train)

#mostrando as estatísticas do processo
print(modelo_cross.sprint_statistics())

#testando o melhor modelo
y_cv = modelo_cross.predict(X_test)
acc = accuracy_score(y_test, y_cv)
print("Acuracia: %.3f" % acc)
print(modelo_cross.leaderboard())

In [ ]:
#17
#usando holdout iterative fit, com base de treinamento em 7 por 3 nos argumentos
modelo_holdout_iterativo = AutoSklearnClassifier(time_left_for_this_task=5*60, per_run_time_limit=30, n_jobs=-1, resampling_strategy = 'holdout-iterative-fit', resampling_strategy_arguments={'train_size': 0.7})
#inicia a busca
modelo_holdout_iterativo.fit(X_train, y_train)

#mostrando as estatísticas do processo
print(modelo_holdout_iterativo.sprint_statistics())

#testando o melhor modelo
y_h2 = modelo_holdout_iterativo.predict(X_test)
acc = accuracy_score(y_test, y_h2)
print("Acuracia: %.3f" % acc)

In [ ]:
#18
#Questão 1 Letra D
#como o melhor resultado normalmente é o cross validation, vou usa-lo aqui
from sklearn.metrics import precision_score

#matrix de confusao
cm = confusion_matrix(y_test, y_cv)
print('matriz de confusao: \n\n' + str(cm))

acc = accuracy_score(y_test, y_cv)
print("\nAcuracia: %.3f" % acc)

pre = precision_score(y_test, y_cv, average = 'macro')
print("Precisao: %.3f" % pre)

A partir desses dados (precisão e acurácia) é possível ver uma eficácia considerável do programa, principalmente após o tratamento de dados, e a entrega de mais instâncias após o oversampling. Nesse exemplo eu também testei antes com undersampling, mas o resultado foi horrível, e quando troquei houve uma mudança significativa.

2) Utilize a mesma base de dados (lembrem de tirar o rótulo, obviamente) da questão anterior de forma que:

A) Execute o K-means e Hierárquico.

B) Teste o K igual à 5 e 7.

C) Na execução do Hierárquico, varie 2 métodos do linkage; OBS.: utilize os mesmos valores de clusters escolhidos na questão anterior.

D) Por fim, faça uma comparação entre os 2 resultados das execuções anteriores e adote uma medida de avaliação própria para clusterização.


In [ ]:
#retirando a 'GRADE' da base
base_final2 = base_final.drop('GRADE', axis = 1)

In [ ]:
#19
#Questão 2 (execute o bloco 11)

#hierárquico com método ward e 3 clusters

from sklearn.cluster import AgglomerativeClustering
import numpy as np

agg_clustering1 = AgglomerativeClustering(n_clusters = 3, affinity = 'euclidean', linkage = 'ward')
predic1 = agg_clustering1.fit_predict(base_final2)

In [ ]:
#20
#hierárquico com método average e 3 clusters

from sklearn.cluster import AgglomerativeClustering
import numpy as np

agg_clustering2 = AgglomerativeClustering(n_clusters = 3, affinity = 'euclidean', linkage = 'average')
predic2 = agg_clustering2.fit_predict(base_final2)

In [ ]:
#21
#K means com 5 clusters
from sklearn.cluster import KMeans
import numpy as np

kmeans1 = KMeans(n_clusters=5, random_state=1)
predic3 = kmeans1.fit_predict(base_final2)

In [ ]:
#23
#K means com 7 clusters
from sklearn.cluster import KMeans
import numpy as np

kmeans2 = KMeans(n_clusters=7, random_state=1)
predic4 = kmeans2.fit_predict(base_final2)

In [ ]:
#25
#Questão 2 Letra D
from sklearn.metrics import silhouette_score
from sklearn.metrics import confusion_matrix

print("Analisando kmeans com 5 clusters:")
pontuacao = silhouette_score(base_final2, kmeans1.labels_, metric = 'euclidean')
print("Silhouette Score com 5 clusters: {}" .format(pontuacao))
print("matriz de confusão")
confusion_matrix(base_final['GRADE'], predic3)

Analisando kmeans com 5 clusters:
Silhouette Score com 5 clusters: 0.12147057948957606
matriz de confusão


array([[10,  3,  8,  1,  0,  0,  0,  0],
       [ 4,  5,  7,  4,  2,  0,  0,  0],
       [ 9,  5,  4,  0,  4,  0,  0,  0],
       [ 0,  5,  2,  8,  7,  0,  0,  0],
       [ 1, 12,  3,  0,  6,  0,  0,  0],
       [ 2,  9,  0, 10,  1,  0,  0,  0],
       [ 5,  4,  4,  5,  4,  0,  0,  0],
       [ 3,  8,  5,  4,  2,  0,  0,  0]])

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import confusion_matrix

print("Analisando kmeans com 7 clusters:")
pontuacao = silhouette_score(base_final2, kmeans2.labels_, metric = 'euclidean')
print("Silhouette Score com 7 clusters: {}" .format(pontuacao))
print("matriz de confusão")
confusion_matrix(base_final['GRADE'], predic4)

Analisando kmeans com 7 clusters:
Silhouette Score com 7 clusters: 0.13677802744151032
matriz de confusão


array([[ 0,  1,  8,  0,  3, 10,  0,  0],
       [ 2,  4,  8,  3,  0,  1,  4,  0],
       [ 3,  6,  6,  1,  3,  2,  1,  0],
       [ 3,  6,  2,  3,  0,  0,  8,  0],
       [ 6,  3,  3,  5,  1,  0,  4,  0],
       [ 1,  7,  0,  4,  2,  0,  8,  0],
       [ 5,  3,  4,  0,  6,  0,  4,  0],
       [ 3,  5,  5,  0,  2,  1,  6,  0]])

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import confusion_matrix

print("Analisando hierarquico ward com 3 clusters:")
pontuacao = silhouette_score(base_final2, agg_clustering1.labels_, metric = 'euclidean')
print("Silhouette Score com 7 clusters: {}" .format(pontuacao))
print("matriz de confusão")
confusion_matrix(base_final['GRADE'], predic1)

Analisando hierarquico ward com 3 clusters:
Silhouette Score com 7 clusters: 0.11234879199194113
matriz de confusão


array([[10,  4,  8,  0,  0,  0,  0,  0],
       [ 7,  6,  9,  0,  0,  0,  0,  0],
       [12,  2,  8,  0,  0,  0,  0,  0],
       [11,  9,  2,  0,  0,  0,  0,  0],
       [ 4,  9,  9,  0,  0,  0,  0,  0],
       [13,  9,  0,  0,  0,  0,  0,  0],
       [10,  4,  8,  0,  0,  0,  0,  0],
       [11,  6,  5,  0,  0,  0,  0,  0]])

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import confusion_matrix

print("Analisando hierarquico average com 3 clusters:")
pontuacao = silhouette_score(base_final2, agg_clustering2.labels_, metric = 'euclidean')
print("Silhouette Score com 3 clusters: {}" .format(pontuacao))
print("matriz de confusão")
confusion_matrix(base_final['GRADE'], predic2)

Analisando hierarquico average com 3 clusters:
Silhouette Score com 3 clusters: 0.19650483524479262
matriz de confusão


array([[18,  4,  0,  0,  0,  0,  0,  0],
       [22,  0,  0,  0,  0,  0,  0,  0],
       [22,  0,  0,  0,  0,  0,  0,  0],
       [20,  0,  2,  0,  0,  0,  0,  0],
       [22,  0,  0,  0,  0,  0,  0,  0],
       [22,  0,  0,  0,  0,  0,  0,  0],
       [22,  0,  0,  0,  0,  0,  0,  0],
       [21,  0,  1,  0,  0,  0,  0,  0]])

É possível observar, a partir do silhouette score, que por ele estar próximo de zero, os clusters estão se sobrepondo. Por estar mais próximo de 1, o método linkage com o linkage em 'average' se mostrou o melhor resultado.